In [75]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import yfinance as yf
from sklearn.preprocessing import LabelEncoder
from datetime import datetime

def corrigir_dataframe(df):
    """Corrige problemas com MultiIndex e garante formato consistente"""
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.get_level_values(0)
    return df

def processar_dados(ticker):
    """Obtém e processa dados do Yahoo Finance"""
    try:
        dados = yf.download(ticker + '.SA', period='3mo', progress=False)
        dados = corrigir_dataframe(dados)
        
        if dados.empty:
            raise ValueError("Nenhum dado obtido")
            
        # Renomear colunas e garantir formato correto
        dados = dados.rename(columns={
            'Open': 'open',
            'High': 'high', 
            'Low': 'low',
            'Close': 'close',
            'Volume': 'volume'
        })
        
        # Verificar colunas essenciais
        colunas_necessarias = ['open', 'high', 'low', 'close', 'volume']
        if not all(col in dados.columns for col in colunas_necessarias):
            missing = [col for col in colunas_necessarias if col not in dados.columns]
            raise ValueError(f"Colunas faltantes: {missing}")
        
        # Adicionar colunas extras
        dados['adjusted_close'] = dados['close'].astype(float)
        dados['dividend_amount'] = 0.0
        dados['split_coefficient'] = 1.0
        dados['ticker'] = ticker + '.SA'
        
        return dados.reset_index().rename(columns={'Date': 'date'})
    
    except Exception as e:
        print(f"Erro ao processar {ticker}: {str(e)}")
        return None

def criar_features(dados, features_esperadas):
    """Cria features de forma segura e compatível"""
    if dados is None or dados.empty:
        return None
        
    df = dados.copy()
    
    try:
        # 1. Corrigir formato do DataFrame
        df = corrigir_dataframe(df)
        
        # 2. Features básicas
        setores = {'PETR4.SA': 'Energia', 'VALE3.SA': 'Mineração', 'ITUB4.SA': 'Financeiro'}
        df['sector'] = df['ticker'].map(setores)
        le = LabelEncoder()
        df['sector_encoded'] = le.fit_transform(df['sector'].fillna('Outros')).astype(float)
        
        # 3. Cálculo de retornos
        for window in [1, 5, 21, 63]:
            df[f'retorno_{window}D'] = df.groupby('ticker')['close'].pct_change().rolling(window).mean().values
        
        # 4. Volatilidade
        df['volatilidade_21D'] = df.groupby('ticker')['retorno_1D'].rolling(21).std().values
        
        # 5. Volume (cálculo seguro)
        df['volume_avg_21D'] = df.groupby('ticker')['volume'].rolling(21).mean().values
        df['volume_spike'] = np.where(
            df['volume_avg_21D'] > 0,
            df['volume'] / df['volume_avg_21D'],
            0
        )
        
        # 6. Features que não podem ser calculadas com 3 meses
        for feature in ['RSI_14', 'BB_upper', 'BB_lower', 'SMA_200']:
            if feature in features_esperadas:
                df[feature] = 0.0
                
        # 7. SMA_50 (se necessário)
        if 'SMA_50' in features_esperadas:
            df['SMA_50'] = df.groupby('ticker')['close'].rolling(50).mean().values
        
        # 8. Features de dividendos
        df['dividend_yield'] = 0.0
        df['dividend_payment'] = 0.0
        
        # 9. Features de setor (simplificadas)
        df['sector_avg_retorno_21D'] = df.groupby('sector')['retorno_21D'].transform('mean')
        df['sector_avg_volume'] = df.groupby('sector')['volume'].transform('mean')
        
        # Garantir todas as features esperadas
        for feature in features_esperadas:
            if feature not in df.columns:
                df[feature] = 0.0
        
        return df[features_esperadas]
    
    except Exception as e:
        print(f"Erro ao criar features: {str(e)}")
        return None

def prever_acao(ticker):
    try:
        # 1. Carregar modelo e features
        modelo = lgb.Booster(model_file='utils/modelo_lgbm_sem_talib.txt')
        features_esperadas = pd.read_csv('utils/features_utilizadas.csv', header=None)[0].tolist()
        
        # Remover possível cabeçalho incorreto
        if features_esperadas[0] == '0':
            features_esperadas = features_esperadas[1:]
        
        print(f"\nFeatures esperadas ({len(features_esperadas)}): {features_esperadas}")
        
        # 2. Obter dados
        dados = processar_dados(ticker)
        if dados is None:
            return None
            
        print("\nColunas disponíveis:", dados.columns.tolist())
        
        # 3. Criar features
        dados_features = criar_features(dados, features_esperadas)
        if dados_features is None:
            return None
            
        print("\nFeatures criadas:", dados_features.columns.tolist())
        
        # 4. Fazer previsão
        ultimo_dia = dados_features.iloc[[-1]].values.astype(float)
        retorno = float(modelo.predict(ultimo_dia)[0])
        
        # 5. Resultado
        print(f"\nPrevisão para {ticker}:")
        print(f"Data: {dados['date'].iloc[-1].strftime('%d/%m/%Y')}")
        print(f"Preço: R${dados['close'].iloc[-1]:.2f}")
        print(f"Retorno previsto: {retorno:.2%}")
        print(f"Preço projetado: R${dados['close'].iloc[-1] * (1 + retorno):.2f}")
        
        return retorno
        
    except Exception as e:
        print(f"Erro ao prever {ticker}: {str(e)}")
        return None

if __name__ == "__main__":
    acoes = ['PETR4']
    resultados = []
    
    for acao in acoes:
        print(f"\nProcessando {acao}...")
        retorno = prever_acao(acao)
        if retorno is not None:
            resultados.append({
                'ticker': acao,
                'data': datetime.now().strftime('%Y-%m-%d'),
                'retorno_previsto': retorno,
                'preco_atual': yf.Ticker(acao + '.SA').history(period='1d')['Close'].iloc[-1]
            })
    
    if resultados:
        pd.DataFrame(resultados).to_csv('previsoes.csv', index=False)
        print("\nPrevisões salvas em 'previsoes.csv'")
    else:
        print("\nNenhuma previsão concluída com sucesso")


Processando PETR4...

Features esperadas (17): ['retorno_1D', 'retorno_5D', 'retorno_21D', 'retorno_63D', 'volatilidade_21D', 'volume_avg_21D', 'volume_spike', 'RSI_14', 'SMA_50', 'SMA_200', 'BB_upper', 'BB_lower', 'dividend_yield', 'dividend_payment', 'sector_encoded', 'sector_avg_retorno_21D', 'sector_avg_volume']

Colunas disponíveis: ['date', 'close', 'high', 'low', 'open', 'volume', 'adjusted_close', 'dividend_amount', 'split_coefficient', 'ticker']

Features criadas: ['retorno_1D', 'retorno_5D', 'retorno_21D', 'retorno_63D', 'volatilidade_21D', 'volume_avg_21D', 'volume_spike', 'RSI_14', 'SMA_50', 'SMA_200', 'BB_upper', 'BB_lower', 'dividend_yield', 'dividend_payment', 'sector_encoded', 'sector_avg_retorno_21D', 'sector_avg_volume']

Previsão para PETR4:
Data: 25/07/2025
Preço: R$31.98
Retorno previsto: 6.13%
Preço projetado: R$33.94

Previsões salvas em 'previsoes.csv'
